# Examples of compartmental models
Using the nifty little functions we created in the file `utils.py`. 

This will look a lot like the Colab notebook we created.

In [ ]:
import utils

import matplotlib.pyplot as plt

In [ ]:
# Test out the CompartmentalModel class
# With two populations: deer and grass

# Define the starting populations in a dictionary
deer_grass_populations = {"deer": [100], "grass": [3000]}

# Define some functions that are the components of the model
def deer_eat_grass(state_dictionary, timestep):
    # timestep is a parameter for consistency with other 
    # functions that might depend on time (say, something cyclical)
    # This function takes the current state of the system and returns
    # the change in the populations coming from deer eating grass
    
    # The deer eat 10% of the grass every year
    # And grow by as much grass as they ate
    new_deer = state_dictionary["grass"] * 0.1
    eaten_grass = -new_deer
    return {"deer": new_deer, "grass": eaten_grass}

def grass_grows(state_dictionary, timestep):
    # The grass grows logistically with a carrying capacity of 3000 and a rate of 0.8
    return {"deer": 0, "grass": 0.8 * state_dictionary["grass"] * (1 - state_dictionary["grass"]) / 3000}

# Compartments are on a new line just for readability
dumb_deer_grass_model = utils.CompartmentalModel(start_states=deer_grass_populations, 
                                                 compartments=[deer_eat_grass, grass_grows],
                                                 time_units="years")

# Run the model for 14 years
dumb_deer_grass_model.simulate(timesteps=14)

# Check that it worked
dumb_deer_grass_model.print_states()

In [ ]:
# Plot

# Grab each set of states
all_states = dumb_deer_grass_model.get_states()
deer_states = all_states["deer"]
grass_states = all_states["grass"]

# Get the timestamps
timestamps = [i for i in range(len(deer_states))]

# Plot the deer and grass populations over time
# `scatter` plots points instead of a smooth line like `plot`
plt.scatter(timestamps, deer_states, label="Deer")
plt.scatter(timestamps, grass_states, label="Grass")

# Add some labels
plt.title("Simple Deer - Grass Model")
plt.xlabel(f"Time ({dumb_deer_grass_model.get_time_units()})")
plt.ylabel("Population")
plt.legend()

plt.show()
plt.close()